In [ ]:
# NYT Developers Call Limit

# there are two rate limits per API: 4,000 requests per day and 10 requests per minute. 

# You should sleep 6 seconds between calls to avoid hitting the per minute rate limit.

In [ ]:
# Sign up for an API key here               https://developer.nytimes.com

In [ ]:

import pandas as pd
import numpy as np

import requests

from datetime import timedelta, date

import time

import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import register_matplotlib_converters

import PIL.Image
from pylab import *
from wordcloud import WordCloud, STOPWORDS


In [ ]:
# ENTER YOUR OWN NYT DEVELOPER API KEY HERE


api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'           # <------------------ ENTER API KEY HERE

In [ ]:
# pull lists from nyt api

lists = requests.get('https://api.nytimes.com/svc/books/v3//lists/names.json?api-key=' + api_key)

print("Please show me 200...", lists.status_code)

     
# Wait for 70 seconds

time.sleep(70)

In [ ]:
# put into dataframe

lists_data = lists.json()['results']
lists_data = pd.DataFrame.from_dict(lists_data)
lists_data

In [ ]:
# get a list of dates... enter relevant dates into start_date and stop_date variable

from datetime import timedelta, date


dates = []

start_date = date(2011, 2, 13)                         # <------------------ ENTER START DATE HERE
end_date = date(2020, 2, 2)                            # <------------------ ENTER END DATE HERE

delta = timedelta(days=7)

while start_date <= end_date:
    
    dates.append(start_date.strftime("%Y-%m-%d"))
    start_date += delta


In [ ]:
# group dates into chunks of 10 for the api call size    (maximum is 10 a minute, 4000 a day)


def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


chunks = []

for group in chunker(dates, 10):
   
    chunks.append(group)
 
    

In [ ]:
# type in the name of the best sellers list you would wish to query make sure to type it in correctly 
# you can find all options from the list_name_encoded column in the 'list' dataframe above
# however please choose only the lists that are updated WEEKLY as that is how I set up the code


list_name = 'combined-print-and-e-book-fiction'       # <------------------ ENTER LIST NAME HERE

In [ ]:
# api call create and append dataframe


api_url_start = 'https://api.nytimes.com/svc/books/v3/lists/'       # generate generic api url
api_url_end = ('/' + list_name  + '.json?api-key=')                 # add in list name



genre_df = pd.DataFrame()                                           # create empty dataframe


for chunk in chunks:                                                # pull 10 dates at a time from total dates
    
    
    for date in chunk:                                              # pull one date at a time from the 10
    
        url = (api_url_start +  date   + api_url_end)               # put togehter api url parts
        response = requests.get(url + api_key)                      # pull from api with key
    
        genre = response.json()                                     # create variable for json file
        genre = pd.DataFrame.from_dict(genre)                       # create dataframe from json file 
        
        books_df = pd.DataFrame.from_dict(genre.iloc[1,4])          # select only book info from dataframe
        
        books_df['bestsellers-date'] = genre.iloc[9,4]              # create and join column for bestsellers dates
        
        genre_df = genre_df.append(books_df, ignore_index=True, sort=True)      # append 15 books to dataframe

        
    
    
        # select only relevant columns add bestsellers date (you may wish to add or delete some)  

        genre_df = genre_df[['bestsellers-date', 'author', 'description', 'price', 'publisher', 'rank', 'rank_last_week', 'title', 'weeks_on_list' ]]
    
    
    
# Wait for 70 seconds to not max out the api call limit  ...and repeat for the next 10 dates

    time.sleep(70)


# show dataframe    

genre_df


In [ ]:
# save dateframe to csv file  (UNHASH THE LINE)

genre_df.to_csv('combined_print_and_e_book_fiction.csv')
        

In [ ]:
# see all column choices

genre_df.columns

In [ ]:
genre_df = pd.read_csv('combined_print_and_e_book_fiction.csv')

In [ ]:
# rank and plot the weeks on the list 

popular = genre_df['title'].value_counts()

popular = pd.DataFrame(popular).reset_index()

popular = popular.rename(columns={"index": "title", "title": "count"})

popular = popular.loc[popular['count']>=52]

popular

In [ ]:
# Data to plot... make sure explode has same number of 

labels = popular['title']
sizes = popular['count']
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightblue','lightsteelblue','silver', 'lightskyblue', 'lightblue','lightsteelblue','silver', 'lightskyblue', 'lightblue','lightsteelblue','silver', 'lightskyblue', 'lightblue']
explode = (0.3, 0.2, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)



# show values instead of percentages
def absolute_value(val):
    a  = sizes[ np.abs(sizes - val/100.*sizes.sum()).idxmin() ]
    return a


plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct=absolute_value, shadow=True, startangle=90)


plt.axis('equal')
plt.title('Books with More than 52 Weeks on the Fiction Bestsellers List', fontsize=22)
plt.show()

In [ ]:
# rank and plot the most popular book authors by weeks on the list 

popular = genre_df['author'].value_counts()

popular = pd.DataFrame(popular).reset_index()

popular = popular.rename(columns={"index": "author", "author": "count"})

popular = popular.loc[popular['count']>=100]

popular

In [ ]:
# Data to plot... (make sure explode has same number of elements as your popular list)

labels = popular['author']
sizes = popular['count']
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightblue','lightsteelblue','silver', 'lightskyblue', 'lightblue','lightsteelblue','silver', 'lightskyblue', 'lightblue','lightsteelblue','silver']
explode = (0.3, 0.2, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)



# show values instead of percentages
def absolute_value(val):
    a  = sizes[ np.abs(sizes - val/100.*sizes.sum()).idxmin() ]
    return a


plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct=absolute_value, shadow=True, startangle=90)


plt.axis('equal')
plt.title('Authors with More than 100 Weeks on the Fiction Bestsellers List', fontsize=22)
plt.show()

In [ ]:
# drop duplicate titles and keep the first occurance 
# (sometimes books get republished and the descriptions get updated so we want the first editions)

titles = genre_df.drop_duplicates(subset='title', keep='first')


In [ ]:
# create wordbank from descriptions

wordbank = titles['description'].unique()

wordbank 

In [ ]:
# build a word cloud

stopwords = set(STOPWORDS)

wordcloud = WordCloud(background_color='white', stopwords=stopwords, max_words=1000, width=2000, height=1000).generate(' '.join(wordbank))

# display the word cloud

fig = plt.figure()
fig.set_figwidth(20) # set width
fig.set_figheight(20) # set height

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# add the uninteresting words to stopwords

stopwords.add('novel')
stopwords.add("series")
stopwords.add("book")
stopwords.add("author")
stopwords.add("writer")
stopwords.add("trilogy")
stopwords.add("stories")
stopwords.add("sequel")
stopwords.add("story")
stopwords.add("Originally published")
stopwords.add("installment")
stopwords.add("set")
stopwords.add("sets")
stopwords.add("Stone Barrington")
stopwords.add("continue")
stopwords.add("must")
stopwords.add("called")
stopwords.add("Jack Reacher")
stopwords.add("will")

# re-generate the word cloud

wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=1000, width=2000, height=1000).generate(' '.join(wordbank))

fig = plt.figure()
fig.set_figwidth(20) # set width
fig.set_figheight(20) # set height

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# open image (just make sure it's in your working directory) and save mask to book

book = np.array(PIL.Image.open('book.png'))

imshow(book)

print('Image opened and saved!')

In [ ]:
# display mask image

fig = plt.figure()
fig.set_figwidth(14) # set width
fig.set_figheight(18) # set height

plt.imshow(book, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# instantiate a word cloud object
book_wc = WordCloud(background_color='white', max_words=500, mask=book, stopwords=stopwords)

# generate the word cloud
book_wc.generate(' '.join(wordbank))

# display the word cloud
fig = plt.figure()
fig.set_figwidth(20) # set width
fig.set_figheight(20) # set height

plt.imshow(book_wc, interpolation='bilinear')
plt.axis('off')
plt.show()
